In [3]:
from Dissects.io import (load_NDskl, 
                         load_image,
                         load_skeleton,
                         save_skeleton,
                         save_fits,
                         save_image,
                        )
from Dissects.image import (z_project,
                            thinning,
                            dilation)
from Dissects.geometry import Skeleton


from Dissects.analysis.analysis import (general_analysis,
                              cellstats
                              )

from Dissects.analysis.analysis_3D_apical import (junction_intensity,
                                                  morphology_analysis)

from Dissects.segmentation.seg_3D_apical import (generate_segmentation)


In [25]:
import os

from skimage import io
import numpy as np
import pandas as pd
import copy
import sys
from skimage import morphology

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go

sys.setrecursionlimit(10000)

import plotly.express as px

from scipy.ndimage.morphology import (binary_fill_holes,
                                      binary_dilation,
                                     )
from scipy import ndimage
import scipy as sci
# %matplotlib notebook
%load_ext snakeviz

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [27]:
import matplotlib as mpl

# update fig parameter
mpl.rcParams['figure.dpi'] = 150 #300
mpl.rcParams['figure.figsize'] = (10,10)

In [17]:
directory="/media/admin-suz/Datas/Comparative-analysis_Folds/T2-T3/FastAiSc_1160x1160"

filename="C1-T2-T3_leg_FastAiSc_1160x1160_px0v09-GB2.fits_c1.35e+03.up.NDskl.a.NDskl"

In [18]:
# get original image
img0 = load_image(os.path.join(directory, 
                              "C1-T2-T3_leg_FastAiSc_1160x1160_px0v09-GB2.tif"))

In [31]:
cp, fil, point, specs = load_NDskl(os.path.join(directory, filename))

# #create skeleton object
# skel = Skeleton(cp, fil, point, specs)

In [20]:
#Clean skeleton
skel.remove_lonely_cp()
skel.remove_free_filament()

/home/admin-suz/miniconda3/lib/python3.8/site-packages/Dissects-0.0.1-py3.8.egg/Dissects/geometry/skeleton.py:60: UserWarning: specs dictionnary is not updated, ncrit value is not updated.
  warnings.warn(


In [21]:
skel.specs

{'ndims': 3,
 'bbox': array([0., 0., 0.]),
 'bbox_delta': array([ 271., 1148.,  106.]),
 'ncrit': 2439,
 'nfil': 2270}

In [22]:
skeleton_image = skel.create_binary_image()
img_binary_3d = morphology.skeletonize(skeleton_image)

# Global variable

Ce n'est pas idéal, mais c'est une solution temporaire pour choisir tout les paramètres au début de l'analyse sans avoir à les chercher dans les différentes cellules...

In [81]:
# Pixel/Voxel size
X_SIZE = 0.0916155
Y_SIZE = 0.0916155
Z_SIZE = 0.2201818

# Save face, edge, vert dataframe from initial segmentation
SAVE_INIT_SEGMENTATION = True
# Save face, edge, vert image from initial segmentation
SAVE_INIT_SEGMENTATION_IMAGE = True

DIRECT_VISUALISATION_VERTEX_EDGES = True
DIRECT_VISUALISATION_FACE = True

# Analyse morphologique
MORPHOLOGICAL_ANALYSIS = True
AREA = True
PERIMETER = True
NB_NEIGHBOR = True
ANISO = True

In [82]:
X_SHAPE = img0[0].shape[2]
Y_SHAPE = img0[0].shape[1]
Z_SHAPE = img0[0].shape[0]


# 3D vertex detection

In [11]:
%pdb

Automatic pdb calling has been turned ON


In [83]:
face_df, edge_df, vert_df, points_df = generate_segmentation(skel, clean=False, **{"X_SIZE":X_SIZE,
                                                                                   "Y_SIZE":Y_SIZE,
                                                                                   "Z_SIZE":Z_SIZE})

if SAVE_INIT_SEGMENTATION : 
    vert_df.to_csv(os.path.join(directory, 'vert_df.csv'))
    edge_df.to_csv(os.path.join(directory, 'edge_df.csv'))
    face_df.to_csv(os.path.join(directory, 'face_df.csv'))


there is a problem


In [ ]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    
    # Vertex image
    vertex_image = np.zeros([Z_SHAPE, Y_SHAPE, X_SHAPE])
    s = ndimage.generate_binary_structure(3,3)

    vertex_image[vert_df.z_pix.to_numpy(),
                 vert_df.y_pix.to_numpy(),
                 vert_df.x_pix.to_numpy()] = 1
    # Dilate a little bit vertex to have a better viewing
    vertex_image = ndimage.morphology.binary_dilation(vertex_image, structure=s)
    
    save_image(vertex_image, 'output_vertex.tif', directory, **{"x_size":X_SIZE,
                                                                "y_size":Y_SIZE,
                                                                "z_size":Z_SIZE,})
    
    image_skeleton = skel.create_binary_image()
    image_skeleton = ndimage.morphology.binary_dilation(image_skeleton, structure=s)
    
    save_image(image_skeleton, 'output_skeleton.tif', directory, **{"x_size":X_SIZE,
                                                                    "y_size":Y_SIZE,
                                                                    "z_size":Z_SIZE,})

In [ ]:
khkhgd

In [ ]:
if DIRECT_VISUALISATION_VERTEX_EDGES: 
    # plot skeleton
    image_skeleton = skel.create_binary_image()
    z0, y0,x0 = np.where(image_skeleton>0)
    background_skeleton = go.Scatter3d(x=x0,
                                        y=y0,
                                        z=z0,
                                        mode='markers',
                                        marker=dict(
                                            size=2,
                                            color='black',
                                            opacity=0.1
                                         )
                        )
    
    
    vertex = go.Scatter3d( x=vert_df.x_pix,
                           y=vert_df.y_pix,
                           z=vert_df.z_pix,
                           mode='markers',
                           marker=dict(
                            size=2,
                            color='red', 
                            opacity=0.5
                         )
                )
    
    fig = go.Figure(data=[background_skeleton, vertex], )
    fig.update_layout(title='Background-vertex', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )
    
    # Edges
    zs,ys,xs = vert_df.loc[edge_df['srce']][['z_pix', 'y_pix', 'x_pix']].values.flatten(order='F').reshape(3,edge_df.shape[0])
    zt,yt,xt = vert_df.loc[edge_df['trgt']][['z_pix', 'y_pix', 'x_pix']].values.flatten(order='F').reshape(3,edge_df.shape[0])

    rand = (np.random.rand(len(zs),3)*255).astype('int')
    rand[0] = 0
    cmap_rand = ListedColormap(rand)
    for i in range(len(zs)):
        fig.add_trace(
                go.Scatter3d(
                    x=[xs[i], xt[i]],
                    y=[ys[i], yt[i]],
                    z=[zs[i], zt[i]],
                    mode='lines',
                    line={"color":rand[[i,i,i]],
                          "width":10,
                         },                
                )
            )
        
    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)
    fig.show()

In [ ]:
import random 
if DIRECT_VISUALISATION_FACE:
    rand = (np.random.rand(face_df.shape[0],3)*255).astype('int')
    rand[0] = 0
    cmap_rand = ListedColormap(rand)


    plot_face=[]
    
    fill_colors=list(mpl.colors.cnames.values())
    random.shuffle(fill_colors)
    for f in (np.unique(edge_df.face)[1:]):

        try:
            edges= edge_df[edge_df.face==f]
            nb_vert = len(edges)
            vert_order = [edges.iloc[0].srce]
            for i in range(nb_vert):
                vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
            zs=vert_df.loc[vert_order].z_pix.to_numpy()
            ys=vert_df.loc[vert_order].y_pix.to_numpy()
            xs=vert_df.loc[vert_order].x_pix.to_numpy()
            plot_face.append(dict(
                type='scatter3d',
                mode='lines',
                x=xs,
                y=ys,
                z=zs,
                name='',
                surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
                surfacecolor=fill_colors[f],
                line=dict(
                    color='black',
                    width=4
                ),
            ))
        except:
            print(f)



    plot_face.append(background_skeleton)
    plot_face.append(vertex)
    fig = go.Figure(data=plot_face, )
    fig.update_layout(title='Filament', 
                      autosize=False,
                      width=1000,
                      height=1000,
                      margin=dict(l=65, r=50, b=65, t=90),
                     )

    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)
    fig.show()

In [192]:
face_df[face_df.columns[[3,4,5,6,7,8,9,10,11,13]]]

,area_approximate,area,perimeter_approximate,perimeter,nb_neighbor,orient_x,orient_y,orient_z,aniso,intensity
-1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,8.608674,60.006726,17.217348,51.278522,4.0,3.160215,139.746342,1.884946,2.439879,-1.0
2,7.515024,34.165299,15.030047,26.166044,5.0,22.079614,74.772619,11.196538,1.374697,-1.0
3,7.557821,43.430373,15.115642,39.209387,4.0,11.410936,157.652294,25.266454,2.793773,-1.0
4,8.986573,47.404843,17.973145,35.879659,6.0,43.857512,63.503674,6.415913,1.351629,-1.0
...,...,...,...,...,...,...,...,...,...,...
128,8.763661,41.927946,17.527321,34.445387,5.0,8.838429,108.576721,51.518931,2.103192,-1.0
129,8.698510,52.333506,17.397021,44.154260,4.0,0.671012,277.053975,135.869393,5.385687,-1.0
130,4.028307,26.503245,8.056614,32.749247,3.0,31.364768,110.040854,15.514907,2.015341,-1.0
131,11.127625,72.712143,22.255251,47.342385,7.0,7.400175,69.436741,-42.288794,1.424831,-1.0


# Analyse

In [84]:
# morphometry analysis

if MORPHOLOGICAL_ANALYSIS :
    morphology_analysis(face_df, 
                        edge_df, 
                        vert_df, 
                        points_df, 
                        area=AREA,
                        perimeter=PERIMETER,
                        nb_neighbor=NB_NEIGHBOR,
                        aniso=ANISO,
                       )

## Mesure d'intensité au niveau des jonctions

In [151]:

# import image original
# get original image

img_myosin = io.imread(os.path.join(directory, 
                              "C2-T2-T3_leg_FastAiSc_1160x1160_px0,09.tif"))


junction_intensity(img_myosin, edge_df, 'myosin_intensity')

0
'Series' object has no attribute 'points'
1
'Series' object has no attribute 'points'
2
'Series' object has no attribute 'points'
3
'Series' object has no attribute 'points'
4
'Series' object has no attribute 'points'
5
'Series' object has no attribute 'points'
6
'Series' object has no attribute 'points'
7
'Series' object has no attribute 'points'
8
'Series' object has no attribute 'points'
9
'Series' object has no attribute 'points'
10
'Series' object has no attribute 'points'
11
'Series' object has no attribute 'points'
12
'Series' object has no attribute 'points'
13
'Series' object has no attribute 'points'
14
'Series' object has no attribute 'points'
15
'Series' object has no attribute 'points'
16
'Series' object has no attribute 'points'
17
'Series' object has no attribute 'points'
18
'Series' object has no attribute 'points'
19
'Series' object has no attribute 'points'
20
'Series' object has no attribute 'points'
21
'Series' object has no attribute 'points'
22
'Series' object h

In [152]:
# plot prot intensity

import plotly.graph_objects as go

z0, y0,x0 = np.where(img_binary_3d==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=0.1
                     )
    )


vertex_isolated = go.Scatter3d(x=vert_df.x,
                      y=vert_df.y,
                      z=vert_df.z,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='red', 
                        opacity=1
                     )
    )



fig = go.Figure(data=[vertex_isolated], )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )



zs,ys,xs = vert_df.loc[edge_df['srce']][list('zyx')].values.flatten(order='F').reshape(3, edge_df.shape[0])
zt,yt,xt = vert_df.loc[edge_df['trgt']][list('zyx')].values.flatten(order='F').reshape(3, edge_df.shape[0])
c = (edge_df.length_approximate/np.max(edge_df.length_approximate)*255).astype('int')

import pylabedge_df as pl
cmap = pl.cm.cividis


for i in range(len(zs)):
    
    fig.add_trace(
            go.Scatter3d(
                x=[xs[i], xt[i]],
                y=[ys[i], yt[i]],
                z=[zs[i], zt[i]],
                mode='lines',
                line={"color":[cmap(c[i]), cmap(c[i])],
                      "width":10,
                     },                
            )
        )


fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [147]:
# plot area

import plotly.graph_objects as go

import pylab as pl
cmap = pl.cm.cividis


c = ((face_df.area[1:]-face_df.area.min())/np.max(face_df.area[1:])*255).astype('int')

plot_face=[]
for f in (np.unique(edge_df.face)[1:]):

    edges= edge_df[edge_df.face==f]
    nb_vert = len(edges)
    vert_order = [edges.iloc[0].srce]
    for i in range(nb_vert):
        vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
    zs=vert_df.loc[vert_order].z.to_numpy()
    ys=vert_df.loc[vert_order].y.to_numpy()
    xs=vert_df.loc[vert_order].x.to_numpy()
    plot_face.append(dict(
        type='scatter3d',
        mode='lines',
        x=xs,
        y=ys,
        z=zs,
        name='',
        surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
        surfacecolor='rgb'+str(cmap(c[f])[:3]),
#             surfacecolor=fill_colors[int(ar[f]/2)],
        line=dict(
            color='black',
            width=4
        ),
    ))
   
    
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [116]:
dfsdfsd

NameError: name 'dfsdfsd' is not defined

# Analyse au niveau de la face

In [160]:
from Dissects.analysis.analysis_3D_apical import face_intensity, compute_normal
from Dissects.utils.utils import pixel_to_um


In [277]:
edge_df['x_um'] = edge_df['x']
edge_df['y_um'] = edge_df['y']
edge_df['z_um'] = edge_df['z']

vert_df['x_um'] = vert_df['x']
vert_df['y_um'] = vert_df['y']
vert_df['z_um'] = vert_df['z']

vert_df['x'] = vert_df['x_pix']
vert_df['y'] = vert_df['y_pix']
vert_df['z'] = vert_df['z_pix']

# global_temp = []

# for i in edge_df.index:
#     x,y,z = edge_df.loc[i][['point_x', 'point_y', 'point_z']]
#     tmp = '['+str(x[0])+','+str(y[0])+','+str(z[0])+']'
        
#     global_temp.append(tmp)
# edge_df['points'] = global_temp

edge_df['points'] = edge_df['filaments']

In [278]:
# C'est long, il ne faut pas s'inquieter... 2-3min... faut apprendre la patience... 

# define pixel size [x,y,z] in µm

pixel_size = {'x':1, 'y':1, 'z':1}
# Largeur d'épaississement de la surface de la cellule
# Cette valeur est ajoutée au dessus et en dessous du plan de la face
thicken = 0.2
# compute_normal(face_df, edge_df, vert_df)
face_intensity(img_myosin, face_df, edge_df, vert_df, thicken, pixel_size)


index 507 is out of bounds for axis 1 with size 2


/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning:

Mean of empty slice.

/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



index 587 is out of bounds for axis 1 with size 2


/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning:

Mean of empty slice.

/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



index 610 is out of bounds for axis 1 with size 2


...

/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning:

Mean of empty slice.

/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



index 208 is out of bounds for axis 1 with size 2


/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning:

Mean of empty slice.

/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



index 587 is out of bounds for axis 1 with size 2


/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning:

Mean of empty slice.

/home/admin-suz/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



In [279]:
face_df

,fx,fy,fz,area_approximate,area,perimeter_approximate,perimeter,nb_neighbor,orient_x,orient_y,orient_z,aniso,115,intensity,intensity_norm
1,20.453160,51.121449,2.972454,8.608674,60.006726,17.217348,51.278522,4.0,3.160215,139.746342,1.884946,2.439879,0.0,NaN,NaN
2,20.521872,55.702224,3.214654,7.515024,34.165299,15.030047,26.166044,5.0,22.079614,74.772619,11.196538,1.374697,0.0,NaN,NaN
3,21.598354,59.481363,3.522909,7.557821,43.430373,15.115642,39.209387,4.0,11.410936,157.652294,25.266454,2.793773,0.0,NaN,NaN
4,18.185677,47.853830,3.376121,8.986573,47.404843,17.973145,35.879659,6.0,43.857512,63.503674,6.415913,1.351629,0.0,NaN,NaN
5,17.755084,42.253069,3.787127,10.972354,75.573541,21.944708,46.919246,5.0,15.032541,90.265822,-1.209736,1.597273,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,8.588953,85.660493,18.220044,8.698510,52.333506,17.397021,44.154260,4.0,0.671012,277.053975,135.869393,5.385687,0.0,NaN,NaN
130,8.917242,88.164649,20.183332,4.028307,26.503245,8.056614,32.749247,3.0,31.364768,110.040854,15.514907,2.015341,0.0,NaN,NaN
131,8.638033,79.103440,16.356362,11.127625,72.712143,22.255251,47.342385,7.0,7.400175,69.436741,-42.288794,1.424831,0.0,NaN,NaN
115,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.000000,5.000000,5.000000,5.000000,5.0,NaN,NaN


In [250]:
# plot area

import plotly.graph_objects as go

import pylab as pl
cmap = pl.cm.cividis
# face_df['intensity']-=face_df.intensity.min()

face_df['intensity_norm']= ((face_df.intensity[1:])/np.max(face_df.intensity[1:])*255).astype('int')


plot_face=[]
for f in (np.unique(edge_df.face)[1:]):
#     print('rgb'+str(cmap(int(ar[f]))))
    edges= edge_df[edge_df.face==f]
    nb_vert = len(edges)
    vert_order = [edges.iloc[0].srce]
    for i in range(nb_vert):
        vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
    zs=vert_df.loc[vert_order].z.to_numpy()
    ys=vert_df.loc[vert_order].y.to_numpy()
    xs=vert_df.loc[vert_order].x.to_numpy()
    plot_face.append(dict(
        type='scatter3d',
        mode='lines',
        x=xs,
        y=ys,
        z=zs,
        name='',
        surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
#         surfacecolor='rgb'+str(cmap(c[f])[:3]),
        surfacecolor='rgb'+str(cmap(face_df['intensity_norm'].iloc[f-1])[0:3]),
#             surfacecolor=fill_colors[int(ar[f]/2)],
        line=dict(
            color='black',
            width=4
        ),
    ))
   
    
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

ValueError: Cannot convert non-finite values (NA or inf) to integer

> /home/admin-suz/miniconda3/lib/python3.8/site-packages/pandas/core/dtypes/cast.py(1068)astype_nansafe()
   1066 
   1067         if not np.isfinite(arr).all():
-> 1068             raise ValueError("Cannot convert non-finite values (NA or inf) to integer")
   1069 
   1070     elif is_object_dtype(arr):

ipdb> c


In [ ]:
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap
rand = (np.random.rand(200,3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)



z0, y0,x0 = np.where(img_binary_3d==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=1
                     )
    )

z0, y0,x0 = np.where(img_plane==1)
face = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='blue', 
                        opacity=0.2
                     )
    )


plot_face=[]
plot_face.append(fond)
plot_face.append(face)
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
from tyssue import Sheet
from tyssue import SheetGeometry
face_df = pd.DataFrame(index=np.arange(1, np.max(edge_df.face)))
# remove_edge = edge_df[edge_df.face==-1].index
# edge_df.drop(remove_edge, axis=0, inplace=True)
# edge_df.reset_index(inplace=True, drop=True)
data={'vert':vert_df, 'edge':edge_df, 'face':face_df}
sheet = Sheet('spherical', data)
sheet.reset_index()
sheet.reset_topo()
SheetGeometry.update_all(sheet)

from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet,list('yz'), mode='2D')

In [ ]:
from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet, mode='3D')
fig